## 대회 Overview

- 제공되는 데이터는 `분석` 관점에서 해볼 것이 많음
    - 어떤 시점 이후로 부동산 가격이 높은지
    - 정책적인 측면과 관련이 있는지
    - ... 등

- 결측치와 이상치를 어떻게 처리할 것인가!
    - 작전 세력, 탈세 등의 거래 -> 이상치 유발

- 연속형/범주형 변수 처리

- 파생 변수 생성
    - 부동산의 도메인을 통해 파생변수 생성


주요점
 - y(target)을 위주로 보는 것은 매우 중요함!
    - train 과 test 가 다른 분포를 갖는 경우 정규화를 시켜줄 필요가 있음


## 데이터 전처리
> 데이터의 특성을 이해, 데이터의 수정 -> `데이터의 품질 향상`

1. 데이터 분석 (EDA)
    - 데이터가 어떤 의미를 가지는지 분석 
    - 다양한 가설 검정
    - 데이터의 이상유무 체크
    - 시각화를 통한 인사이트 생성

2. 결측치 처리

3. 이상치 처리

4. 연속형 변수 처리

5. 범주형 변수 처리

6. 파생 변수 생성 (Feature Engineering)

7. 변수 선택

### 데이터셋 개요

In [ ]:
!pip install eli5==0.13.0

# 한글 폰트 사용을 위한 라이브러리입니다.
!apt-get install -y fonts-nanum

In [2]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

import eli5
from eli5.sklearn import PermutationImportance

In [5]:
!pwd

/root/workspace


In [10]:
# 필요한 데이터를 load 하겠습니다. 경로는 환경에 맞게 지정해주면 됩니다.
train_path = './data/train.csv'
test_path  = './data/test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)

# 추가 데이터 load
bus_feature = pd.read_csv('./data/bus_feature.csv')
subway_feature = pd.read_csv('./data/subway_feature.csv')

In [11]:
# Train data와 Test data shape은 아래와 같습니다.
print('Train data shape : ', dt.shape, 'Test data shape : ', dt_test.shape)
print('Bus feature data shape : ', bus_feature.shape, 'Subway feature data shape : ', subway_feature.shape)

Train data shape :  (1118822, 52) Test data shape :  (9272, 51)
Bus feature data shape :  (12584, 6) Subway feature data shape :  (768, 5)


In [12]:
# Train과 Test data를 살펴보겠습니다.
display(dt.head(1))
display(dt_test.head(1))      # 부동산 실거래가(=Target) column이 제외된 모습입니다.
display(bus_feature.head(1))
display(subway_feature.head(1))      # 부동산 실거래가(=Target) column이 제외된 모습입니다.

,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일,target
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,201712,8,3,1987,...,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0,124000


,시군구,번지,본번,부번,아파트명,전용면적(㎡),계약년월,계약일,층,건축년도,...,청소비관리형태,건축면적,주차대수,기타/의무/임대/임의=1/2/3/4,단지승인일,사용허가여부,관리비 업로드,좌표X,좌표Y,단지신청일
0,서울특별시 강남구 개포동,658-1,658.0,1.0,개포6차우성,79.97,202307,26,5,1987,...,직영,4858.0,262.0,임의,2022-11-17 13:00:29.0,Y,N,127.05721,37.476763,2022-11-17 10:19:06.0


,노드 ID,정류소번호,정류소명,X좌표,Y좌표,정류소 타입
0,100000001,1001,종로2가사거리,126.987752,37.569808,중앙차로


,역사_ID,역사명,호선,위도,경도
0,9996,미사,5호선,37.560927,127.193877


In [85]:
dt.columns

Index(['시군구', '번지', '본번', '부번', '아파트명', '전용면적(㎡)', '계약년월', '계약일', '층', '건축년도',
       '도로명', '해제사유발생일', '등기신청일자', '거래유형', '중개사소재지', 'k-단지분류(아파트,주상복합등등)',
       'k-전화번호', 'k-팩스번호', '단지소개기존clob', 'k-세대타입(분양형태)', 'k-관리방식', 'k-복도유형',
       'k-난방방식', 'k-전체동수', 'k-전체세대수', 'k-건설사(시공사)', 'k-시행사', 'k-사용검사일-사용승인일',
       'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', 'k-전용면적별세대현황(60㎡이하)',
       'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', 'k-135㎡초과', 'k-홈페이지',
       'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태',
       '건축면적', '주차대수', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드',
       '좌표X', '좌표Y', '단지신청일', 'target'],
      dtype='object')

## EDA

`Trash Features`
- 'k-전화번호', 'k-팩스번호', 'k-홈페이지', 'k-관리방식', 'k-시행사', 'k-사용검사일-사용승인일', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '사용허가여부'

`연관성 있어보이는 Features`
- '번지', '본번', '부번'
- '계약년월', '계약일'
- '단지신청일' - '단지승인일' ?
- 'k-연면적', 'k-주거전용면적', 'k-관리비부과면적', '건축면적'

In [86]:
dt = dt.drop(columns=['k-전화번호', 'k-팩스번호', 'k-홈페이지', 'k-관리방식', 'k-시행사', 'k-사용검사일-사용승인일', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '사용허가여부'])
dt_test = dt_test.drop(columns=['k-전화번호', 'k-팩스번호', 'k-홈페이지', 'k-관리방식', 'k-시행사', 'k-사용검사일-사용승인일', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '사용허가여부'])

In [87]:
print('Train data shape : ', dt.shape, 'Test data shape : ', dt_test.shape)

Train data shape :  (1118822, 41) Test data shape :  (9272, 40)


`시군구`
- 서울특별시 데이터이므로 **시** 는 의미없다.
- **동** 단위로 구분하는 경우 데이터 수가 너무 없는 경우가 존재한다.
    - 주요한 **동**을 구반하는 flag 를 두고 기본적으로 **구** 단위로 구분하는 것이 좋아보인다.

In [89]:
dt['시군구']

0          서울특별시 강남구 개포동
1          서울특별시 강남구 개포동
2          서울특별시 강남구 개포동
3          서울특별시 강남구 개포동
4          서울특별시 강남구 개포동
               ...      
1118817    서울특별시 은평구 구산동
1118818    서울특별시 은평구 구산동
1118819    서울특별시 은평구 구산동
1118820    서울특별시 은평구 구산동
1118821     서울특별시 중구 묵정동
Name: 시군구, Length: 1118822, dtype: object

In [91]:
# Null data 없음
# dt['시군구'].isnull().sum() >>> 0

# 구, 동 정보 추가
dt['구'] = dt['시군구'].apply(lambda x: x.split()[1])
dt['동'] = dt['시군구'].apply(lambda x: x.split()[2])